## PyCity Schools Analysis
By creating the pandas analyses below, multiple summaries of the data showcase: 
1.	District Summary; this shows an aggregate of all the:
    a.	Schools and students within the district
    b.	Total district budget
    c.	Average math and reading scores
    d.	Percentage passing math, reading, and overall 
2.	School Summary; similar to the district summary utilizes all the same data points, but by each individual high school (Bailey, Cabrera, Ford, etc.) 
3.	Highest & Lowest Performing School Summaries; which showcases the top schools in the district as well as the lowest schools in the district based on percentage of students passing overall (i.e., both reading and math >= 70)
4.	Math, Reading, and Overall Scores by Grade Summaries; which shows the individual schools average math and reading scores, as well as percentage of students passing math, reading, and overall for each grade (9th to 12th) 
5.	Scores by School Spending, School Size, and School Type Summaries; which cuts the data to showcase how the spending, size, and type (i.e., district or charter) may impact the average grades and passing percentage of each school 

Looking at the available data, you can make some comparisons: 
1.	Looking at school spending compared to average grades, you’ll notice that while it may be assumed school that spend more will result in better grades, in fact those schools who are spending <$585 per student see higher passing rates (~90.4%) to those spending $645-$680 (~53.5%)
2.	Instead, it’s more likely that the size of the school or the type of school has a higher impact than spending; as small (<1000 total students) and medium sized (1000-2000 total students) schools tend to have better passing rates than large sized schools (>2000 total students)
3.	Similarly, charter schools tend to have higher passing rates (~90.4%) than district schools  (~53.7%)

Therefore, if you’re looking to improve scores and passing rates, rather than increasing budget of schools, it may make more sense to open more charter schools in order to lessen the amount of students in a single school and thereby increase the amount of small or medium-sized schools. 



## Setup

In [340]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [341]:
# Calculate the total number of unique schools
school_count = school_data_complete["school_name"].nunique()
school_count


15

In [342]:
# Calculate the total number of students
student_count = school_data_complete["student_name"].nunique()
student_count


32715

In [343]:
# Calculate the total budget
school_budgets = school_data_complete.groupby("school_name")["budget"].unique().sum()
total_budget = school_budgets[0]
total_budget


24649428

In [344]:
# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].mean()
average_math_score


78.98537145774827

In [345]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score


81.87784018381414

In [346]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage


89.77533241632278

In [347]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)
passing_reading_count = school_data_complete[(school_data_complete["reading_score"]>=70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage


102.73574812777014

In [348]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate


78.031484028733

In [349]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame ({
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "Passing Math Percent": [passing_math_percentage],
    "Passing Reading Percent": [passing_reading_percentage],
    "Overall Passing Percent": [overall_passing_rate]
})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math Percent,Passing Reading Percent,Overall Passing Percent
0,15,"32,715","$24,649,428.00",78.985371,81.87784,89.775332,102.735748,78.031484


## School Summary

In [350]:
# Use the code provided to select the type per school from school_data
type_per_school = school_data.set_index(["school_name"])["type"]
type_per_school

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [351]:
# Calculate the total student count per school from school_data
per_school_counts = school_data_complete.groupby("school_name")["student_name"].count()
per_school_counts


school_name
Bailey High School       4976
Cabrera High School      1858
Figueroa High School     2949
Ford High School         2739
Griffin High School      1468
Hernandez High School    4635
Holden High School        427
Huang High School        2917
Johnson High School      4761
Pena High School          962
Rodriguez High School    3999
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: student_name, dtype: int64

In [352]:
# Calculate the total school budget and per capita spending per school from school_data
per_school_budget = school_data_complete.groupby("school_name")["budget"].mean()
per_school_capita = per_school_budget / per_school_counts
per_school_budget


school_name
Bailey High School       3124928.0
Cabrera High School      1081356.0
Figueroa High School     1884411.0
Ford High School         1763916.0
Griffin High School       917500.0
Hernandez High School    3022020.0
Holden High School        248087.0
Huang High School        1910635.0
Johnson High School      3094650.0
Pena High School          585858.0
Rodriguez High School    2547363.0
Shelton High School      1056600.0
Thomas High School       1043130.0
Wilson High School       1319574.0
Wright High School       1049400.0
Name: budget, dtype: float64

In [353]:
# Calculate the average test scores per school from school_data_complete
per_school_math = school_data_complete.groupby("school_name")["math_score"].mean()
per_school_reading = school_data_complete.groupby("school_name")["reading_score"].mean()


In [354]:
# Calculate the number of students per school with math scores of 70 or higher from school_data_complete
# Filters the dataset to include only students passing math with scores of 70 or higher
students_passing_math = school_data_complete[school_data_complete["math_score"] >= 70]

# Group the filtered dataset by school name and count the number of students passing math for each school
students_passing_math_per_school = students_passing_math.groupby("school_name")["student_name"].count()
students_passing_math_per_school

school_name
Bailey High School       3318
Cabrera High School      1749
Figueroa High School     1946
Ford High School         1871
Griffin High School      1371
Hernandez High School    3094
Holden High School        395
Huang High School        1916
Johnson High School      3145
Pena High School          910
Rodriguez High School    2654
Shelton High School      1653
Thomas High School       1525
Wilson High School       2143
Wright High School       1680
Name: student_name, dtype: int64

In [355]:
# Calculate the number of students per school with reading scores of 70 or higher from school_data_complete
students_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70]

# Group the filtered dataset by school name and count the number of students passing math for each school
students_passing_reading_per_school = students_passing_reading.groupby("school_name")["student_name"].count()
students_passing_reading_per_school


school_name
Bailey High School       4077
Cabrera High School      1803
Figueroa High School     2381
Ford High School         2172
Griffin High School      1426
Hernandez High School    3748
Holden High School        411
Huang High School        2372
Johnson High School      3867
Pena High School          923
Rodriguez High School    3208
Shelton High School      1688
Thomas High School       1591
Wilson High School       2204
Wright High School       1739
Name: student_name, dtype: int64

In [356]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()
school_students_passing_math_and_reading


school_name
Bailey High School       2719
Cabrera High School      1697
Figueroa High School     1569
Ford High School         1487
Griffin High School      1330
Hernandez High School    2481
Holden High School        381
Huang High School        1561
Johnson High School      2549
Pena High School          871
Rodriguez High School    2119
Shelton High School      1583
Thomas High School       1487
Wilson High School       2068
Wright High School       1626
dtype: int64

In [357]:
# Use the provided code to calculate the passing rates
per_school_passing_math = students_passing_math_per_school / per_school_counts * 100
per_school_passing_reading = students_passing_reading_per_school / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100


In [358]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame ({
    "School Type": type_per_school,
    "Total Student Counts": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Scores": per_school_math,
    "Average Reading Scores": per_school_reading,
    "Total Students Passing Math": students_passing_math_per_school,
    "Total Students Passing Reading": students_passing_reading_per_school,
    "Total Students Passing Math & Reading": school_students_passing_math_and_reading,
    "Percentage Students Passing Math": per_school_passing_math,
    "Percentage Students Passing Reading": per_school_passing_reading,
    "Percentage Students Passing Overall": overall_passing_rate
})

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary


,School Type,Total Student Counts,Total School Budget,Per Student Budget,Average Math Scores,Average Reading Scores,Total Students Passing Math,Total Students Passing Reading,Total Students Passing Math & Reading,Percentage Students Passing Math,Percentage Students Passing Reading,Percentage Students Passing Overall
school_name,,,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,3318,4077,2719,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,1749,1803,1697,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,1946,2381,1569,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,1871,2172,1487,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,1371,1426,1330,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,3094,3748,2481,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,395,411,381,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,1916,2372,1561,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,3145,3867,2549,66.057551,81.222432,53.539172


## Highest-Performing Schools (by % Overall Passing)

In [359]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values("Percentage Students Passing Overall", ascending=False)
top_schools.head(5)


,School Type,Total Student Counts,Total School Budget,Per Student Budget,Average Math Scores,Average Reading Scores,Total Students Passing Math,Total Students Passing Reading,Total Students Passing Math & Reading,Percentage Students Passing Math,Percentage Students Passing Reading,Percentage Students Passing Overall
school_name,,,,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,1749,1803,1697,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,1525,1591,1487,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,1371,1426,1330,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,2143,2204,2068,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,910,923,871,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [360]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values("Percentage Students Passing Overall", ascending=True)
bottom_schools.head(5)


,School Type,Total Student Counts,Total School Budget,Per Student Budget,Average Math Scores,Average Reading Scores,Total Students Passing Math,Total Students Passing Reading,Total Students Passing Math & Reading,Percentage Students Passing Math,Percentage Students Passing Reading,Percentage Students Passing Overall
school_name,,,,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,2654,3208,2119,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,1946,2381,1569,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,1916,2372,1561,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,3094,3748,2481,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,3145,3867,2549,66.057551,81.222432,53.539172


## Math Scores by Grade

In [361]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grader_math_scores = ninth_graders.groupby("school_name")["math_score"].mean()
tenth_grader_math_scores = tenth_graders.groupby("school_name")["math_score"].mean()
eleventh_grader_math_scores = eleventh_graders.groupby("school_name")["math_score"].mean()
twelfth_grader_math_scores = twelfth_graders.groupby("school_name")["math_score"].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame ({ 
    "Ninth Grade Avg. Math Scores": ninth_grader_math_scores,
    "Tenth Grade Avg. Math Scores": tenth_grader_math_scores,
    "Eleventh Grade Avg. Math Scores": eleventh_grader_math_scores,
    "Twelfth Grade Avg. Math Scores": twelfth_grader_math_scores,
})

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade


,Ninth Grade Avg. Math Scores,Tenth Grade Avg. Math Scores,Eleventh Grade Avg. Math Scores,Twelfth Grade Avg. Math Scores
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [362]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grader_reading_scores = ninth_graders.groupby("school_name")["reading_score"].mean()
tenth_grader_reading_scores = tenth_graders.groupby("school_name")["reading_score"].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby("school_name")["reading_score"].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby("school_name")["reading_score"].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame ({ 
    "9th Avg. Reading Scores": ninth_grader_reading_scores,
    "10th Avg. Reading Scores": tenth_grader_reading_scores,
    "11th Avg. Reading Scores": eleventh_grader_reading_scores,
    "12th Avg. Reading Scores": twelfth_grader_reading_scores,
})

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th Avg. Reading Scores", "10th Avg. Reading Scores", "11th Avg. Reading Scores", "12th Avg. Reading Scores"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade


,9th Avg. Reading Scores,10th Avg. Reading Scores,11th Avg. Reading Scores,12th Avg. Reading Scores
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [363]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Convert the bin values to integers, help from Xpert Learning Assistant below
spending_bins = [int(x) for x in spending_bins]

In [364]:
# Create a copy of the school summary since it has the "Per Student Budget"
school_spending_df = per_school_summary.copy()

In [365]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_summary["Per Student Budget"], bins=spending_bins, labels=spending_labels)


TypeError: '<' not supported between instances of 'int' and 'str'

In [ ]:
#  Calculate averages for the desired columns.
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Scores"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Scores"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Percentage Students Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Percentage Students Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Percentage Students Passing Overall"].mean()

C:\Users\lexis\AppData\Local\Temp\ipykernel_40772\1749006152.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Scores"].mean()
C:\Users\lexis\AppData\Local\Temp\ipykernel_40772\1749006152.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Scores"].mean()
C:\Users\lexis\AppData\Local\Temp\ipykernel_40772\1749006152.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True 

In [ ]:
# Assemble into DataFrame
spending_summary = pd.DataFrame ({ 
    "Avg. Math Scores": spending_math_scores,
    "Avg. Reading Scores": spending_reading_scores,  
    "Percent Passing Math": spending_passing_math,
    "Percent Passing Reading": spending_passing_reading,
    "Percent Passing Overall": overall_passing_spending,   
})

# Display results
spending_summary


,Avg. Math Scores,Avg. Reading Scores,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Student Counts"], bins=size_bins, labels=size_labels)


In [ ]:
# Calculate averages for the desired columns.
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Scores"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Scores"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["Percentage Students Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["Percentage Students Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["Percentage Students Passing Overall"].mean()


C:\Users\lexis\AppData\Local\Temp\ipykernel_40772\3922849865.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Scores"].mean()
C:\Users\lexis\AppData\Local\Temp\ipykernel_40772\3922849865.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Scores"].mean()
C:\Users\lexis\AppData\Local\Temp\ipykernel_40772\3922849865.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame ({ 
    "Avg. Math Scores": size_math_scores,
    "Avg. Reading Scores": size_reading_scores,  
    "Percent Passing Math": size_passing_math,
    "Percent Passing Reading": size_passing_reading,
    "Percent Passing Overall": size_overall_passing   
})

# Display results
size_summary


,Avg. Math Scores,Avg. Reading Scores,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Scores"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Scores"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["Percentage Students Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["Percentage Students Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["Percentage Students Passing Overall"].mean()

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame ({ 
    "Avg. Math Scores": average_math_score_by_type,
    "Avg. Reading Scores": average_reading_score_by_type,  
    "Percent Passing Math": average_percent_passing_math_by_type,
    "Percent Passing Reading": average_percent_passing_reading_by_type,
    "Percent Passing Overall": average_percent_overall_passing_by_type   
})

# Display results
type_summary


,Avg. Math Scores,Avg. Reading Scores,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
